## Get version of local si446x library in use

In [1]:
import datetime
print(datetime.datetime.now())

2017-04-30 17:57:04.932991


In [2]:
from si446x.si446xvers import __version__
print(__version__)

0.1.53


## Open Radio Driver Dbus Interface

In [3]:
%run Radio_DBus_IO.ipynb
radio = si446x_dbus_open()

In [4]:
help(radio)

Help on <CompositeObject> object:

class <CompositeObject>(org.tagnet.si446x, org.freedesktop.DBus.Properties, org.freedesktop.DBus.Introspectable, org.freedesktop.DBus.Peer, org.freedesktop.DBus.ObjectManager)
 |  Method resolution order:
 |      <CompositeObject>
 |      org.tagnet.si446x
 |      org.freedesktop.DBus.Properties
 |      org.freedesktop.DBus.Introspectable
 |      org.freedesktop.DBus.Peer
 |      org.freedesktop.DBus.ObjectManager
 |      pydbus.proxy.ProxyObject
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, iface)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __qualname__ = '<CompositeObject>(org.tagnet.si446x+org.freedesk...top...
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from org.tagnet.si446x:
 |  
 |  cca(...)
 |      cca(self) -> u
 |  
 |  clear_status(...)
 |      clear_status(

In [33]:
help(Trace)

Help on class Trace in module si446x.si446xtrace:

class Trace
 |  Radio Trace Handler
 |  
 |  This class provides control and support functions for low level radio
 |  trace handling. The trace events are added to a circular buffer that
 |  can be displayed when needed.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, size, form='%Y.%m.%d %H:%M:%S', rb=None)
 |      Initialize new Trace buffer
 |      
 |      If no ringbuffer is specified, allocate a new ringbuffer.
 |      Also optionally provide the format for presenting time.
 |  
 |  add(self, where, data, s_name='string', level=1)
 |      add a new trace record.
 |      
 |      A trace record consists of:
 |      -  where   a well known name for the record type, (see RadioTraceIds),
 |      -  data    the content to be added in the trace record
 |      -  s_name  an optional name of the structure used to decode contents, and
 |      -  level   the number of stack frames to include for call history
 |  
 |  display(self, 

In [5]:
def wait_for_status(s):
    while (True):
        status = radio.status()
        if (status.startswith(s) or
          status.startswith('ealready')):
            break
        print(status[:3])

In [6]:
print(radio.control('TURNOFF'))
wait_for_status('OFF')
print(radio.status())

ok 2017.04.30 17:57:07.428024 <States=S_SDN>:None
OFF, <States=S_SDN>, unshuts 0
 RX: packets 0, len_errors 0, timeouts 0, sync_errors 0, crc_errors 0, rssi 0, 
 TX: packets 0, errors 0, timeouts 0, power 5, 
 sync frr: 00000000
 node P211, version 0.1.53 [2017.04.30 17:57:07.452605]


In [7]:
print(radio.control('TURNON'))
wait_for_status('ON')
print(radio.status())

ok 2017.04.30 17:57:07.707001 <States=S_POR_W>:None
OFF
OFF
ON, <States=S_RX_ON>, unshuts 0
 RX: packets 0, len_errors 0, timeouts 0, sync_errors 0, crc_errors 0, rssi 0, 
 TX: packets 0, errors 0, timeouts 0, power 5, 
 sync frr: 08000054
 node P211, version 0.1.53 [2017.04.30 17:57:08.021028]


In [8]:
print(radio.status())

ON, <States=S_RX_ON>, unshuts 0
 RX: packets 0, len_errors 0, timeouts 0, sync_errors 0, crc_errors 0, rssi 0, 
 TX: packets 0, errors 0, timeouts 0, power 5, 
 sync frr: 08000054
 node P211, version 0.1.53 [2017.04.30 17:57:08.063965]


## Get si446x Part Info

In [9]:
part_info = si446x_dbus_part_info(radio)
for s in part_info:
    print(s[0])
s_name='get_gpio_pin_cfg_rsp_s'
my_struct = eval(s_name)
print('\n GPIO Pin Settings:')
print(radio_display_structs[my_struct](my_struct,si446x_dbus_get_gpio_config(radio)))

Part Number: 4463, rev: 0x11, id: 0xf, romid: 0x3
Firmware: 3.0.15, patch: 0x0, func: 0x1

 GPIO Pin Settings:
get_gpio_pin_cfg_rsp_s: ff9c08a120a70b00
, drive_strength HIGH, pin[0](IN_SLEEP:ACTIVE) PULL_DIS, pin[1](CTS:INACTIVE) PULL_DIS, pin[2](RX_STATE:ACTIVE) PULL_DIS, pin[3](TX_STATE:INACTIVE) PULL_DIS, NIRQ ACTIVE / PULL_DIS, SDO INACTIVE / PULL_DIS


## Get ALL Radio Properties

In [10]:
psum = []
total = 0
for gp_n, gp_s in radio_config_groups.iteritems():
    p = si446x_dbus_get_property(radio, radio_config_group_ids.parse(gp_n), 0, gp_s.sizeof())
    psum.append((radio_config_group_ids.parse(gp_n), len(p), hexlify(p)))
    total += len(p)

print('Total length of properties {}\n'.format(total))

for p in psum:
    print(p)

Total length of properties 308

('INT_CTL', 4, '033b2b00')
('GLOBAL', 10, '52001860000001600000')
('FRR_CTL', 4, '0904060a')
('FREQ_CONTROL', 8, '380eeeee444420fe')
('SYNC', 5, '01b42b2dd4')
('PREAMBLE', 14, '0814000f31000000000000000000')
('PKT', 53, '850108ffff0082002a01003030000104a200000000000000000000000000000000000104820081000a000000000000000000000000')
('MODEM_CHFLT', 36, 'a28126af3feec8c7dbf20208070315fc0f00a28126af3feec8c7dbf20208070315fc0f00')
('MODEM', 82, '030007061a8005c9c380000576000067604d362111080301018008038000202000000177015d8600af02c2000436801d1004800000e20000115252001affff002a0ca402d68300ad0180200c25004003000a')
('SYNTH', 8, '2c0e0b040c730305')
('PA', 6, '087f003d0023')
('RX_HOP', 66, '0401000102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f303132333435363738393a3b3c3d3e3f')
('MATCH', 12, '000000000000000000000000')


In [11]:
for gp_n, gp_s in radio_config_groups.iteritems():
    p = si446x_dbus_get_property(radio, radio_config_group_ids.parse(gp_n), 0, gp_s.sizeof())
    print(radio_config_group_ids.parse(gp_n), len(p), hexlify(p))
    print(gp_s.parse(p))
    print('')

('INT_CTL', 4, '033b2b00')
Container:
    enable = Container:
        CHIP_INT_STATUS_EN = False
        MODEM_INT_STATUS_EN = True
        PH_INT_STATUS_EN = True
    ph_pend = Container:
        FILTER_MATCH = False
        FILTER_MISS = False
        PACKET_SENT = True
        PACKET_RX = True
        CRC_ERROR = True
        TX_FIFO_ALMOST_EMPTY = True
        RX_FIFO_ALMOST_FULL = True
    modem_pend = Container:
        POSTAMBLE_DETECT = False
        INVALID_SYNC = True
        RSSI_JUMP = False
        RSSI = True
        INVALID_PREAMBLE = False
        PREAMBLE_DETECT = True
        SYNC_DETECT = True
    chip_pend = Container:
        CAL = False
        FIFO_UNDERFLOW_OVERFLOW_ERROR = False
        STATE_CHANGE = False
        CMD_ERROR = False
        CHIP_READY = False
        LOW_BATT = False
        WUT = False

('GLOBAL', 10, '52001860000001600000')
Container:
    xo_tune = 82
    clk_cfg = Container:
        divided_clk_en = 'DISABLE'
        divided_clk_sel = 'DIV_1

('RX_HOP', 66, '0401000102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f303132333435363738393a3b3c3d3e3f')
Container:
    control = 4
    table_size = 1
    table_entries = '\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./0123456789:;<=>?'

('MATCH', 12, '000000000000000000000000')
Container:
    match_field_s = [
        Container:
            value = 0
            mask = 0
            ctrl = 0
        Container:
            value = 0
            mask = 0
            ctrl = 0
        Container:
            value = 0
            mask = 0
            ctrl = 0
        Container:
            value = 0
            mask = 0
            ctrl = 0
    ]



### Formatted Radio Properties

In [12]:
plist = []
total = 0
for gp_n, gp_s in radio_config_groups.iteritems():
    p = si446x_dbus_get_property(radio, radio_config_group_ids.parse(gp_n), 0, gp_s.sizeof())
    plist.append('\n\n++++  {}  ++++\n'.format(radio_config_group_ids.parse(gp_n)))
    plist.append(gp_s.parse(p))

for p in plist:
    print(p)



++++  INT_CTL  ++++

Container:
    enable = Container:
        CHIP_INT_STATUS_EN = False
        MODEM_INT_STATUS_EN = True
        PH_INT_STATUS_EN = True
    ph_pend = Container:
        FILTER_MATCH = False
        FILTER_MISS = False
        PACKET_SENT = True
        PACKET_RX = True
        CRC_ERROR = True
        TX_FIFO_ALMOST_EMPTY = True
        RX_FIFO_ALMOST_FULL = True
    modem_pend = Container:
        POSTAMBLE_DETECT = False
        INVALID_SYNC = True
        RSSI_JUMP = False
        RSSI = True
        INVALID_PREAMBLE = False
        PREAMBLE_DETECT = True
        SYNC_DETECT = True
    chip_pend = Container:
        CAL = False
        FIFO_UNDERFLOW_OVERFLOW_ERROR = False
        STATE_CHANGE = False
        CMD_ERROR = False
        CHIP_READY = False
        LOW_BATT = False
        WUT = False


++++  GLOBAL  ++++

Container:
    xo_tune = 82
    clk_cfg = Container:
        divided_clk_en = 'DISABLE'
        divided_clk_sel = 'DIV_1'
        clk_32k_sel 

## Dump One Register Group at a time

In [13]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from si446x.si446xdef import *

def fetch_and_decode(group):
    gname = radio_config_group_ids.decoding[group]
    g_s = radio_config_groups[radio_config_group_ids.build(gname)]
    gid = radio_config_group_ids.build(gname)
    p = si446x_dbus_get_property(radio, gname, 0, g_s.sizeof())
    print(gname, len(p), hexlify(p))
    rsp = g_s.parse(p)
    print(radio_display_structs[g_s](g_s, p))

interact(fetch_and_decode, group=radio_config_group_ids.encoding)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.fetch_and_decode>

## Dump Radio Registers/Properties and Read Trace Buffer

In [34]:
from si446x.si446xtrace import * 

# dump current radio register and property values. need to dump the trace
# buffer to see the results.
#
reply = radio.dump_radio(' ')
print(reply)

ok
 node P211, version 0.1.53 [2017.04.30 18:02:32.471266]


In [35]:
time_offset=''
old_time_offset=''

In [36]:
reply = radio.dump_trace(-1)
rb = RingBuffer(len(reply))
for x in reply:
    rb.append(x)
t = Trace(0, rb=rb)
print(t.rb.len())

1000


In [37]:
x = 1000
entries = t.filter(filter='', begin=old_time_offset)
p = t.display(entries)
print('Total number of entries: {}, time offset(old/new): {} / {}\n'.format(len(entries), old_time_offset, time_offset))
for entry in p:
    lst = []
    for x in entry[5].split(' '): lst.append(str(x))
    if (not lst):
        s = ''
    elif (lst[1] == 'string'):
        s = ' '.join(lst[2:])
    else:
        s = lst[1]
    print("{} {:<14} {:<20} {}".format(entry[1],entry[2],entry[3], s))

Total number of entries: 1000, time offset(old/new):  / 

2017.04.30 16:32:51.910382  -5380.560812  RADIO_CMD            get_property_cmd_s
2017.04.30 16:32:51.911547  -5380.559646  RADIO_RSP            get_property_rsp_s
2017.04.30 16:32:51.922307  -5380.548887  RADIO_CMD            get_property_cmd_s
2017.04.30 16:32:51.923252  -5380.547942  RADIO_RSP            get_property_rsp_s
2017.04.30 16:32:51.961862  -5380.509332  RADIO_CMD            get_property_cmd_s
2017.04.30 16:32:51.963059  -5380.508134  RADIO_RSP            get_property_rsp_s
2017.04.30 16:32:51.977657  -5380.493537  RADIO_CMD            get_property_cmd_s
2017.04.30 16:32:51.978915  -5380.492278  RADIO_RSP            get_property_rsp_s
2017.04.30 16:32:51.990161  -5380.481032  RADIO_CMD            get_property_cmd_s
2017.04.30 16:32:51.991152  -5380.480042  RADIO_RSP            get_property_rsp_s
2017.04.30 16:32:52.007180  -5380.464013  RADIO_CMD            get_property_cmd_s
2017.04.30 16:32:52.008358  -5380.462836

2017.04.30 16:32:54.887207  -5377.583987  RADIO_CMD            set_property_cmd_s
2017.04.30 16:32:54.888103  -5377.583091  RADIO_CMD            set_property_cmd_s
2017.04.30 16:32:54.889744  -5377.581450  RADIO_CMD            get_property_cmd_s
2017.04.30 16:32:54.890857  -5377.580336  RADIO_RSP            get_property_rsp_s
2017.04.30 16:32:54.892920  -5377.578274  RADIO_CMD            get_property_cmd_s
2017.04.30 16:32:54.894018  -5377.577175  RADIO_RSP            get_property_rsp_s
2017.04.30 16:32:54.895773  -5377.575420  RADIO_CMD            get_property_cmd_s
2017.04.30 16:32:54.896869  -5377.574324  RADIO_RSP            get_property_rsp_s
2017.04.30 16:32:54.898468  -5377.572726  RADIO_CMD            get_property_cmd_s
2017.04.30 16:32:54.899563  -5377.571631  RADIO_RSP            get_property_rsp_s
2017.04.30 16:32:54.901218  -5377.569975  RADIO_CMD            get_property_cmd_s
2017.04.30 16:32:54.902316  -5377.568878  RADIO_RSP            get_property_rsp_s
2017.04.30 16:32

2017.04.30 17:57:07.738256  -324.732937   RADIO_CMD            set_property_cmd_s
2017.04.30 17:57:07.738884  -324.732309   RADIO_CMD            set_property_cmd_s
2017.04.30 17:57:07.739597  -324.731597   RADIO_CMD            set_property_cmd_s
2017.04.30 17:57:07.740241  -324.730953   RADIO_CMD            set_property_cmd_s
2017.04.30 17:57:07.740923  -324.730270   RADIO_CMD            set_property_cmd_s
2017.04.30 17:57:07.741883  -324.729311   RADIO_CMD            set_property_cmd_s
2017.04.30 17:57:07.742508  -324.728685   RADIO_CMD            set_property_cmd_s
2017.04.30 17:57:07.743341  -324.727852   RADIO_CMD            set_property_cmd_s
2017.04.30 17:57:07.744204  -324.726989   RADIO_CMD            set_property_cmd_s
2017.04.30 17:57:07.745008  -324.726185   RADIO_CMD            set_property_cmd_s
2017.04.30 17:57:07.745635  -324.725559   RADIO_CMD            set_property_cmd_s
2017.04.30 17:57:07.746498  -324.724696   RADIO_CMD            set_property_cmd_s
2017.04.30 17:57

2017.04.30 17:57:08.387140  -324.084053   RADIO_RSP            get_property_rsp_s
2017.04.30 17:57:08.397602  -324.073592   RADIO_CMD            get_property_cmd_s
2017.04.30 17:57:08.398492  -324.072702   RADIO_RSP            get_property_rsp_s
2017.04.30 17:57:08.408937  -324.062256   RADIO_CMD            get_property_cmd_s
2017.04.30 17:57:08.410109  -324.061084   RADIO_RSP            get_property_rsp_s
2017.04.30 17:57:08.420475  -324.050718   RADIO_CMD            get_property_cmd_s
2017.04.30 17:57:08.421633  -324.049560   RADIO_RSP            get_property_rsp_s
2017.04.30 17:57:08.432074  -324.039120   RADIO_CMD            get_property_cmd_s
2017.04.30 17:57:08.433235  -324.037958   RADIO_RSP            get_property_rsp_s
2017.04.30 17:57:08.443607  -324.027587   RADIO_CMD            get_property_cmd_s
2017.04.30 17:57:08.444767  -324.026426   RADIO_RSP            get_property_rsp_s
2017.04.30 17:57:08.455216  -324.015977   RADIO_CMD            get_property_cmd_s
2017.04.30 17:57

2017.04.30 17:57:29.059134  -303.412060   RADIO_CMD            get_property_cmd_s
2017.04.30 17:57:29.060014  -303.411179   RADIO_RSP            get_property_rsp_s
2017.04.30 17:57:29.077299  -303.393895   RADIO_CMD            get_property_cmd_s
2017.04.30 17:57:29.078496  -303.392697   RADIO_RSP            get_property_rsp_s
2017.04.30 17:57:29.095177  -303.376016   RADIO_CMD            get_property_cmd_s
2017.04.30 17:57:29.096375  -303.374818   RADIO_RSP            get_property_rsp_s
2017.04.30 17:57:29.107381  -303.363813   RADIO_CMD            get_property_cmd_s
2017.04.30 17:57:29.108529  -303.362665   RADIO_RSP            get_property_rsp_s
2017.04.30 17:57:29.119481  -303.351713   RADIO_CMD            get_property_cmd_s
2017.04.30 17:57:29.120624  -303.350570   RADIO_RSP            get_property_rsp_s
2017.04.30 17:57:29.131179  -303.340015   RADIO_CMD            get_property_cmd_s
2017.04.30 17:57:29.132045  -303.339149   RADIO_RSP            get_property_rsp_s
2017.04.30 17:57

In [19]:
old_time_offset='2017.04.27 21:34:59.375587'
old_time_offset = time_offset
time_offset = p[-1][1]

In [43]:
x = 1000
entries = t.filter(begin='2017.04.30 16:32:52.352425')
p = t.display(entries)
print('Total number of entries: {}, time offset(old/new): {} / {}\n'.format(len(entries), old_time_offset, time_offset))
for entry in p:
    print("{:^10}, {} {} {}".format(entry[0], entry[1],entry[2], entry[3]))
    print(entry[5])

Total number of entries: 952, time offset(old/new):  / 

   4809   , 2017.04.30 16:32:52.352425  -5380.118769 RADIO_INT
 sync frr: 03200052
   4810   , 2017.04.30 16:32:52.353079  -5380.118115 RADIO_CMD
 read_cmd_s command: GET_INT_STATUS
   4811   , 2017.04.30 16:32:52.353375  -5380.117818 RADIO_PEND
 read_cmd_s command: GET_INT_STATUS
   4812   , 2017.04.30 16:32:52.354600  -5380.116593 RADIO_RSP
 int_status_rsp_s 
   4813   , 2017.04.30 16:32:52.354749  -5380.116444 RADIO_PEND
 int_status_rsp_s 
   4814   , 2017.04.30 16:32:52.430497  -5380.040696 RADIO_FRR
 fast_frr_s state: READY, ph_pend:  PACKET_SENT, rssi: 82
   4815   , 2017.04.30 16:32:52.430711  -5380.040482 RADIO_FSM
 string E_WAIT_DONE / S_TX_ACTIVE frr:03200052
   4816   , 2017.04.30 16:32:52.431147  -5380.040047 RADIO_ACTION
 tx timeout <Events=E_WAIT_DONE>
   4817   , 2017.04.30 16:32:52.434932  -5380.036261 RADIO_CMD
 start_rx_cmd_s Container({'next_state1': 'NOCHANGE', 'next_state2': 'READY', 'next_state3': 'READY', '

   4916   , 2017.04.30 16:32:54.946022  -5377.525171 RADIO_CMD
 get_property_cmd_s Container({'num_props': 15, 'cmd': 'GET_PROPERTY', 'group': 'MODEM', 'start_prop': 30})
   4917   , 2017.04.30 16:32:54.947118  -5377.524076 RADIO_RSP
 get_property_rsp_s Container({'data': [32, 32, 0, 0, 1, 119, 1, 93, 134, 0, 175, 2, 194, 0, 4, 0], 'cts': 255})
   4918   , 2017.04.30 16:32:54.949636  -5377.521557 RADIO_CMD
 get_property_cmd_s Container({'num_props': 15, 'cmd': 'GET_PROPERTY', 'group': 'MODEM', 'start_prop': 45})
   4919   , 2017.04.30 16:32:54.950772  -5377.520422 RADIO_RSP
 get_property_rsp_s Container({'data': [54, 128, 29, 16, 4, 128, 0, 0, 226, 0, 0, 17, 82, 82, 0, 0], 'cts': 255})
   4920   , 2017.04.30 16:32:54.953296  -5377.517897 RADIO_CMD
 get_property_cmd_s Container({'num_props': 15, 'cmd': 'GET_PROPERTY', 'group': 'MODEM', 'start_prop': 60})
   4921   , 2017.04.30 16:32:54.954416  -5377.516778 RADIO_RSP
 get_property_rsp_s Container({'data': [26, 255, 255, 0, 42, 12, 164, 2

   5057   , 2017.04.30 17:57:07.745008  -324.726185 RADIO_CMD
 set_property_cmd_s Container({'num_props': 1, 'cmd': 'SET_PROPERTY', 'group': 'MODEM', 'start_prop': 53})
   5058   , 2017.04.30 17:57:07.745635  -324.725559 RADIO_CMD
 set_property_cmd_s Container({'num_props': 9, 'cmd': 'SET_PROPERTY', 'group': 'MODEM', 'start_prop': 56})
   5059   , 2017.04.30 17:57:07.746498  -324.724696 RADIO_CMD
 set_property_cmd_s Container({'num_props': 8, 'cmd': 'SET_PROPERTY', 'group': 'MODEM', 'start_prop': 66})
   5060   , 2017.04.30 17:57:07.747348  -324.723846 RADIO_CMD
 set_property_cmd_s Container({'num_props': 1, 'cmd': 'SET_PROPERTY', 'group': 'MODEM', 'start_prop': 78})
   5061   , 2017.04.30 17:57:07.748024  -324.723169 RADIO_CMD
 set_property_cmd_s Container({'num_props': 1, 'cmd': 'SET_PROPERTY', 'group': 'MODEM', 'start_prop': 81})
   5062   , 2017.04.30 17:57:07.748652  -324.722541 RADIO_CMD
 set_property_cmd_s Container({'num_props': 12, 'cmd': 'SET_PROPERTY', 'group': 'MODEM_CHFLT'

   5227   , 2017.04.30 17:57:08.269907  -324.201286 RADIO_RSP
 get_property_rsp_s Container({'data': [82, 0, 24, 96, 0, 0, 1, 96, 0, 0], 'cts': 255})
   5228   , 2017.04.30 17:57:08.281078  -324.190116 RADIO_CMD
 get_property_cmd_s Container({'num_props': 4, 'cmd': 'GET_PROPERTY', 'group': 'FRR_CTL', 'start_prop': 0})
   5229   , 2017.04.30 17:57:08.281900  -324.189293 RADIO_RSP
 get_property_rsp_s Container({'data': [9, 4, 6, 10], 'cts': 255})
   5230   , 2017.04.30 17:57:08.292293  -324.178901 RADIO_CMD
 get_property_cmd_s Container({'num_props': 8, 'cmd': 'GET_PROPERTY', 'group': 'FREQ_CONTROL', 'start_prop': 0})
   5231   , 2017.04.30 17:57:08.293276  -324.177918 RADIO_RSP
 get_property_rsp_s Container({'data': [56, 14, 238, 238, 68, 68, 32, 254], 'cts': 255})
   5232   , 2017.04.30 17:57:08.303803  -324.167390 RADIO_CMD
 get_property_cmd_s Container({'num_props': 5, 'cmd': 'GET_PROPERTY', 'group': 'SYNC', 'start_prop': 0})
   5233   , 2017.04.30 17:57:08.304663  -324.166530 RADIO_

   5430   , 2017.04.30 17:57:11.020395  -321.450799 RADIO_FSM
 string E_PREAMBLE_DETECT / S_RX_ON frr:08000054
   5431   , 2017.04.30 17:57:11.020909  -321.450285 RADIO_ACTION
 nop <Events=E_PREAMBLE_DETECT>
   5432   , 2017.04.30 17:57:11.023633  -321.447560 RADIO_CMD
 get_clear_int_cmd_s , ph_pend:  FILTER_MATCH FILTER_MISS PACKET_SENT PACKET_RX CRC_ERROR TX_FIFO_ALMOST_EMPTY RX_FIFO_ALMOST_FULL, modem_pend:  POSTAMBLE_DETECT INVALID_SYNC RSSI_JUMP RSSI INVALID_PREAMBLE SYNC_DETECT, chip_pend:  CAL FIFO_UNDERFLOW_OVERFLOW_ERROR STATE_CHANGE CMD_ERROR CHIP_READY LOW_BATT WUT
   5433   , 2017.04.30 17:57:11.024085  -321.447109 RADIO_PEND
 get_clear_int_cmd_s , ph_pend:  FILTER_MATCH FILTER_MISS PACKET_SENT PACKET_RX CRC_ERROR TX_FIFO_ALMOST_EMPTY RX_FIFO_ALMOST_FULL, modem_pend:  POSTAMBLE_DETECT INVALID_SYNC RSSI_JUMP RSSI INVALID_PREAMBLE SYNC_DETECT, chip_pend:  CAL FIFO_UNDERFLOW_OVERFLOW_ERROR STATE_CHANGE CMD_ERROR CHIP_READY LOW_BATT WUT
   5434   , 2017.04.30 17:57:11.025248  -

   5742   , 2017.04.30 18:02:32.457506  -0.013687 RADIO_CMD
 get_clear_int_cmd_s , ph_pend:  FILTER_MATCH FILTER_MISS PACKET_SENT PACKET_RX CRC_ERROR TX_FIFO_ALMOST_EMPTY RX_FIFO_ALMOST_FULL, modem_pend:  POSTAMBLE_DETECT INVALID_SYNC RSSI_JUMP RSSI INVALID_PREAMBLE PREAMBLE_DETECT SYNC_DETECT, chip_pend:  CAL FIFO_UNDERFLOW_OVERFLOW_ERROR STATE_CHANGE CMD_ERROR CHIP_READY LOW_BATT WUT
   5743   , 2017.04.30 18:02:32.458010  -0.013183 RADIO_PEND
 get_clear_int_cmd_s , ph_pend:  FILTER_MATCH FILTER_MISS PACKET_SENT PACKET_RX CRC_ERROR TX_FIFO_ALMOST_EMPTY RX_FIFO_ALMOST_FULL, modem_pend:  POSTAMBLE_DETECT INVALID_SYNC RSSI_JUMP RSSI INVALID_PREAMBLE PREAMBLE_DETECT SYNC_DETECT, chip_pend:  CAL FIFO_UNDERFLOW_OVERFLOW_ERROR STATE_CHANGE CMD_ERROR CHIP_READY LOW_BATT WUT
   5744   , 2017.04.30 18:02:32.459311  -0.011883 RADIO_RSP
 int_status_rsp_s , ph_status:  TX_FIFO_ALMOST_EMPTY, modem_pend:  PREAMBLE_DETECT, modem_status:  RSSI PREAMBLE_DETECT, chip_status:  STATE_CHANGE CHIP_READY
  

In [39]:
x = 1000
entries = t.filter(begin='2017.04.30 16:32:52.352425')
print('Total number of entries: {}\n'.format(len(entries)))
for entry in entries:
    print("{0!s}".format(t.display([entry])))
    x = x - 1
    if (x < 0): break

Total number of entries: 952

[[1953, '2017.04.30 16:32:52.352425', ' -5380.118769', 'RADIO_INT', '  runUntilCurrent:797', ' sync frr: 03200052']]
[[1954, '2017.04.30 16:32:52.353079', ' -5380.118115', 'RADIO_CMD', '  get_clear_interrupts:390 clear_interrupts:219', ' read_cmd_s command: GET_INT_STATUS']]
[[1955, '2017.04.30 16:32:52.353375', ' -5380.117818', 'RADIO_PEND', '  interrupt_handler:436 get_clear_interrupts:390', ' read_cmd_s command: GET_INT_STATUS']]
[[1956, '2017.04.30 16:32:52.354600', ' -5380.116593', 'RADIO_RSP', '  interrupt_handler:436 get_clear_interrupts:391', ' int_status_rsp_s ']]
[[1957, '2017.04.30 16:32:52.354749', ' -5380.116444', 'RADIO_PEND', '  radio_interrupt:314 interrupt_handler:436', ' int_status_rsp_s ']]
[[1958, '2017.04.30 16:32:52.430497', ' -5380.040696', 'RADIO_FRR', '  step_fsm:454 fast_all:311', ' fast_frr_s state: READY, ph_pend:  PACKET_SENT, rssi: 82']]
[[1959, '2017.04.30 16:32:52.430711', ' -5380.040482', 'RADIO_FSM', '  timeout_expired_syn

[[2040, '2017.04.30 16:32:54.904093', ' -5377.567100', 'RADIO_CMD', '  dump_radio:283 get_property:484', " get_property_cmd_s Container({'num_props': 14, 'cmd': 'GET_PROPERTY', 'group': 'PREAMBLE', 'start_prop': 0})"]]
[[2041, '2017.04.30 16:32:54.905190', ' -5377.566003', 'RADIO_RSP', '  dump_radio:283 get_property:485', " get_property_rsp_s Container({'data': [8, 20, 0, 15, 49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'cts': 255})"]]
[[2042, '2017.04.30 16:32:54.908539', ' -5377.562655', 'RADIO_CMD', '  dump_radio:283 get_property:484', " get_property_cmd_s Container({'num_props': 15, 'cmd': 'GET_PROPERTY', 'group': 'PKT', 'start_prop': 0})"]]
[[2043, '2017.04.30 16:32:54.909641', ' -5377.561552', 'RADIO_RSP', '  dump_radio:283 get_property:485', " get_property_rsp_s Container({'data': [133, 1, 8, 255, 255, 0, 130, 0, 42, 1, 0, 48, 48, 0, 1, 0], 'cts': 255})"]]
[[2044, '2017.04.30 16:32:54.914391', ' -5377.556802', 'RADIO_CMD', '  dump_radio:283 get_property:484', " get_property_cmd_s Conta

[[2131, '2017.04.30 16:32:55.060141', ' -5377.411053', 'RADIO_CMD', '  dump_radio:283 get_property:484', " get_property_cmd_s Container({'num_props': 15, 'cmd': 'GET_PROPERTY', 'group': 'RX_HOP', 'start_prop': 0})"]]
[[2132, '2017.04.30 16:32:55.061244', ' -5377.409949', 'RADIO_RSP', '  dump_radio:283 get_property:485', " get_property_rsp_s Container({'data': [4, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0], 'cts': 255})"]]
[[2133, '2017.04.30 16:32:55.062773', ' -5377.408420', 'RADIO_CMD', '  dump_radio:283 get_property:484', " get_property_cmd_s Container({'num_props': 15, 'cmd': 'GET_PROPERTY', 'group': 'RX_HOP', 'start_prop': 15})"]]
[[2134, '2017.04.30 16:32:55.063862', ' -5377.407332', 'RADIO_RSP', '  dump_radio:283 get_property:485', " get_property_rsp_s Container({'data': [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 0], 'cts': 255})"]]
[[2135, '2017.04.30 16:32:55.065387', ' -5377.405806', 'RADIO_CMD', '  dump_radio:283 get_property:484', " get_property_cmd_s

[[2246, '2017.04.30 17:57:07.808593', ' -324.662601', 'RADIO_RSP', '  dump_radio:283 get_property:485', " get_property_rsp_s Container({'data': [7, 3, 21, 252, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'cts': 255})"]]
[[2247, '2017.04.30 17:57:07.810638', ' -324.660555', 'RADIO_CMD', '  dump_radio:283 get_property:484', " get_property_cmd_s Container({'num_props': 15, 'cmd': 'GET_PROPERTY', 'group': 'MODEM', 'start_prop': 0})"]]
[[2248, '2017.04.30 17:57:07.811810', ' -324.659384', 'RADIO_RSP', '  dump_radio:283 get_property:485', " get_property_rsp_s Container({'data': [3, 0, 7, 6, 26, 128, 5, 201, 195, 128, 0, 5, 118, 0, 0, 0], 'cts': 255})"]]
[[2249, '2017.04.30 17:57:07.814338', ' -324.656855', 'RADIO_CMD', '  dump_radio:283 get_property:484', " get_property_cmd_s Container({'num_props': 15, 'cmd': 'GET_PROPERTY', 'group': 'MODEM', 'start_prop': 15})"]]
[[2250, '2017.04.30 17:57:07.815438', ' -324.655755', 'RADIO_RSP', '  dump_radio:283 get_property:485', " get_property_rsp_s Container

[[2375, '2017.04.30 17:57:08.293276', ' -324.177918', 'RADIO_RSP', '  executeMethod:531 dbus_spi_send_recv:243', " get_property_rsp_s Container({'data': [56, 14, 238, 238, 68, 68, 32, 254], 'cts': 255})"]]
[[2376, '2017.04.30 17:57:08.303803', ' -324.167390', 'RADIO_CMD', '  executeMethod:531 dbus_spi_send_recv:242', " get_property_cmd_s Container({'num_props': 5, 'cmd': 'GET_PROPERTY', 'group': 'SYNC', 'start_prop': 0})"]]
[[2377, '2017.04.30 17:57:08.304663', ' -324.166530', 'RADIO_RSP', '  executeMethod:531 dbus_spi_send_recv:243', " get_property_rsp_s Container({'data': [1, 180, 43, 45, 212], 'cts': 255})"]]
[[2378, '2017.04.30 17:57:08.314887', ' -324.156307', 'RADIO_CMD', '  executeMethod:531 dbus_spi_send_recv:242', " get_property_cmd_s Container({'num_props': 14, 'cmd': 'GET_PROPERTY', 'group': 'PREAMBLE', 'start_prop': 0})"]]
[[2379, '2017.04.30 17:57:08.316018', ' -324.155176', 'RADIO_RSP', '  executeMethod:531 dbus_spi_send_recv:243', " get_property_rsp_s Container({'data': 

[[2500, '2017.04.30 17:57:09.295770', ' -323.175423', 'RADIO_CMD', '  executeMethod:531 dbus_spi_send_recv:242', " get_property_cmd_s Container({'num_props': 6, 'cmd': 'GET_PROPERTY', 'group': 'MODEM_CHFLT', 'start_prop': 30})"]]
[[2501, '2017.04.30 17:57:09.296662', ' -323.174532', 'RADIO_RSP', '  executeMethod:531 dbus_spi_send_recv:243', " get_property_rsp_s Container({'data': [7, 3, 21, 252, 15, 0], 'cts': 255})"]]
[[2502, '2017.04.30 17:57:09.307418', ' -323.163776', 'RADIO_CMD', '  executeMethod:531 dbus_spi_send_recv:242', " get_property_cmd_s Container({'num_props': 15, 'cmd': 'GET_PROPERTY', 'group': 'MODEM', 'start_prop': 0})"]]
[[2503, '2017.04.30 17:57:09.308598', ' -323.162595', 'RADIO_RSP', '  executeMethod:531 dbus_spi_send_recv:243', " get_property_rsp_s Container({'data': [3, 0, 7, 6, 26, 128, 5, 201, 195, 128, 0, 5, 118, 0, 0], 'cts': 255})"]]
[[2504, '2017.04.30 17:57:09.319134', ' -323.152059', 'RADIO_CMD', '  executeMethod:531 dbus_spi_send_recv:242', " get_propert

[[2662, '2017.04.30 17:57:29.805537', ' -302.665656', 'RADIO_CMD', '  set_power:593 set_property:584', " set_property_cmd_s Container({'num_props': 1, 'cmd': 'SET_PROPERTY', 'group': 'PA', 'start_prop': 1})"]]
[[2663, '2017.04.30 17:57:29.812828', ' -302.658366', 'RADIO_CMD', '  tx_start:331 start_tx:646', " start_tx_cmd_s Container({'tx_len': 10, 'cmd': 'START_TX', 'condition': Container({'start': 'IMMEDIATE', 'txcomplete_state': 'READY', 'retransmit': 'NO'}), 'channel': 0})"]]
[[2664, '2017.04.30 17:57:29.913866', ' -302.557328', 'RADIO_FRR', '  step_fsm:454 fast_all:311', ' fast_frr_s state: READY, ph_pend:  PACKET_SENT,modem_pend:  RSSI INVALID_PREAMBLE, rssi: 76']]
[[2665, '2017.04.30 17:57:29.914047', ' -302.557147', 'RADIO_FSM', '  timeout_expired_sync:356', bytearray(b' string E_WAIT_DONE / S_TX_ACTIVE frr:03200c4c')]]
[[2666, '2017.04.30 17:57:29.914483', ' -302.556711', 'RADIO_ACTION', '  output:592 output_A_TX_TIMEOUT:161', ' tx timeout <Events=E_WAIT_DONE>']]
[[2667, '2017.

In [22]:
entries = t.filter()
print("rb len: {}, filter len: {}".format(t.rb.len(), len(entries)))

rb len: 1000, filter len: 1000


In [40]:
reply = radio.dump_trace(0)
print(len(reply))
for tim,where,sig,s_name,dat in reply:
    my_struct = None
    if (s_name == 'string'):
        result = bytearray(dat)
    else:
        result = '{:<20} {}'.format(s_name, binascii.hexlify(bytearray(dat[:8])))
        if ((s_name == 'set_property_cmd_s') or (s_name == 'get_property_cmd_s')
           or (s_name == 'get_property_rsp_s')):
            my_struct = None
        else:
            my_struct = eval(s_name)
    f = '{} {:^16} {}'.format(t.format_time(tim),
                      where,
                      result)
    print(f)
    if (my_struct):
        st = radio_display_structs[my_struct](my_struct, bytearray(dat))
        print('   {}\n'.format(st))

1000
2017.04.30 16:32:51.910382    RADIO_CMD     get_property_cmd_s   12120f1e
2017.04.30 16:32:51.911547    RADIO_RSP     get_property_rsp_s   ff00000000010482
2017.04.30 16:32:51.922307    RADIO_CMD     get_property_cmd_s   1212082d
2017.04.30 16:32:51.923252    RADIO_RSP     get_property_rsp_s   ff00000000000000
2017.04.30 16:32:51.961862    RADIO_CMD     get_property_cmd_s   12210f00
2017.04.30 16:32:51.963059    RADIO_RSP     get_property_rsp_s   ffa28126af3feec8
2017.04.30 16:32:51.977657    RADIO_CMD     get_property_cmd_s   12210f0f
2017.04.30 16:32:51.978915    RADIO_RSP     get_property_rsp_s   fffc0f00a28126af
2017.04.30 16:32:51.990161    RADIO_CMD     get_property_cmd_s   1221061e
2017.04.30 16:32:51.991152    RADIO_RSP     get_property_rsp_s   ff070315fc0f00
2017.04.30 16:32:52.007180    RADIO_CMD     get_property_cmd_s   12200f00
2017.04.30 16:32:52.008358    RADIO_RSP     get_property_rsp_s   ff030007061a8005
2017.04.30 16:32:52.018754    RADIO_CMD     get_property_cmd_

2017.04.30 16:32:54.874746    RADIO_CMD     set_property_cmd_s   11200842a402d683
2017.04.30 16:32:54.875586    RADIO_CMD     set_property_cmd_s   1120014e40
2017.04.30 16:32:54.876216    RADIO_CMD     set_property_cmd_s   112001510a
2017.04.30 16:32:54.876844    RADIO_CMD     set_property_cmd_s   11210c00a28126af
2017.04.30 16:32:54.877803    RADIO_CMD     set_property_cmd_s   11210c0c070315fc
2017.04.30 16:32:54.878758    RADIO_CMD     set_property_cmd_s   11210c18c8c7dbf2
2017.04.30 16:32:54.879715    RADIO_CMD     set_property_cmd_s   11220400087f003d
2017.04.30 16:32:54.880436    RADIO_CMD     set_property_cmd_s   112307002c0e0b04
2017.04.30 16:32:54.881246    RADIO_CMD     set_property_cmd_s   11400800380eeeee
2017.04.30 16:32:54.882061    RADIO_CMD     set_property_cmd_s   131c082120000000
2017.04.30 16:32:54.882778    RADIO_CMD     set_property_cmd_s   1100010360
2017.04.30 16:32:54.883409    RADIO_CMD     set_property_cmd_s   11010400033b2b00
2017.04.30 16:32:54.884124    RADI


2017.04.30 16:32:55.075570    RADIO_DUMP    synth_group_s        2c0e0b040c730305
   Container:
    pfdcp_cpff = 44
    pfdcp_cpint = 14
    vco_kv = 11
    lpfilt3 = 4
    lpfilt2 = 12
    lpfilt1 = 115
    lpfilt0 = 3
    vco_kvcal = 5

2017.04.30 16:32:55.075709    RADIO_DUMP    frr_ctl_group_s      0904060a
   CURRENT_STATE  INT_PH_PEND  INT_MODEM_PEND  LATCHED_RSSI

2017.04.30 16:32:55.075846    RADIO_DUMP    modem_group_s        030007061a8005c9
   Container:
    mod_type = Container:
        tx_direct_mode_type = 'SYNC'
        tx_driect_mode_gpio = 'GPIO_1'
        mod_source = 'PACKET'
        mod_type = '_2GFSK'
    map_control = Container:
        enmanch = 'DISABLE'
        eninv_rxbit = 'NO_INVERT'
        eninv_txbit = 'NO_INVERT'
        eninv_fd = 'NO_INVERT_POL'
    dsm_ctrl = Container:
        dsmclk_sel = 'PLL'
        dsm_mode = 'MASH'
        dsmdt_en = 'DISABLE'
        rsmdttp = 'ZERO'
        dsm_rst = 'ENABLE'
        dsm_lsb = 'FORCED_HIGH'
        dsm_order

2017.04.30 17:57:07.848421    RADIO_CMD     get_property_cmd_s   1250063c
2017.04.30 17:57:07.849529    RADIO_RSP     get_property_rsp_s   ff3a3b3c3d3e3f00
2017.04.30 17:57:07.851159    RADIO_CMD     get_property_cmd_s   12300c00
2017.04.30 17:57:07.852260    RADIO_RSP     get_property_rsp_s   ff00000000000000
2017.04.30 17:57:07.854049    RADIO_FRR     fast_frr_s           03000000
   state: READY, rssi: 0

2017.04.30 17:57:07.854224    RADIO_FSM     E_CONFIG_DONE / S_CONFIG_W frr:03000000
2017.04.30 17:57:07.854665   RADIO_ACTION   ready <Events=E_CONFIG_DONE>
2017.04.30 17:57:07.855349    RADIO_CMD     read_cmd_s           20
   command: GET_INT_STATUS

2017.04.30 17:57:07.855678    RADIO_PEND    read_cmd_s           20
   command: GET_INT_STATUS

2017.04.30 17:57:07.857112    RADIO_CMD     get_property_cmd_s   12010400
2017.04.30 17:57:07.858367    RADIO_RSP     get_property_rsp_s   ff033b2b00000000
2017.04.30 17:57:07.860610    RADIO_CMD     get_property_cmd_s   12000a00
2017.04.3

2017.04.30 17:57:08.720180    RADIO_CMD     get_property_cmd_s   12100e00
2017.04.30 17:57:08.721350    RADIO_RSP     get_property_rsp_s   ff0814000f310000
2017.04.30 17:57:08.741992    RADIO_CMD     get_property_cmd_s   12120f00
2017.04.30 17:57:08.743143    RADIO_RSP     get_property_rsp_s   ff850108ffff0082
2017.04.30 17:57:08.753576    RADIO_CMD     get_property_cmd_s   12120f0f
2017.04.30 17:57:08.754736    RADIO_RSP     get_property_rsp_s   ff04a20000000000
2017.04.30 17:57:08.765077    RADIO_CMD     get_property_cmd_s   12120f1e
2017.04.30 17:57:08.766305    RADIO_RSP     get_property_rsp_s   ff00000000010482
2017.04.30 17:57:08.784111    RADIO_CMD     get_property_cmd_s   1212082d
2017.04.30 17:57:08.785121    RADIO_RSP     get_property_rsp_s   ff00000000000000
2017.04.30 17:57:08.823543    RADIO_CMD     get_property_cmd_s   12210f00
2017.04.30 17:57:08.824728    RADIO_RSP     get_property_rsp_s   ffa28126af3feec8
2017.04.30 17:57:08.835299    RADIO_CMD     get_property_cmd_s  

2017.04.30 17:57:30.492291    RADIO_FSM     E_TURNOFF / S_RX_ON frr:08200c48
2017.04.30 17:57:30.492718   RADIO_ACTION   power down <Events=E_TURNOFF>
2017.04.30 17:57:30.493288    RADIO_CHIP    set GPIO pin 18 (SI446x sdn disable)
2017.04.30 17:57:31.520415    RADIO_IOC     TURNON
2017.04.30 17:57:31.520999    RADIO_FRR     fast_frr_s           00000000
   state: NOCHANGE, rssi: 0

2017.04.30 17:57:31.521176    RADIO_FSM     E_TURNON / S_SDN frr:00000000
2017.04.30 17:57:31.521616   RADIO_ACTION   unshutdown <Events=E_TURNON>
2017.04.30 17:57:31.521877    RADIO_GPIO    clear GPIO pin 18 (SI446x sdn enable)
2017.04.30 17:57:31.724972    RADIO_FRR     fast_frr_s           00000000
   state: NOCHANGE, rssi: 0

2017.04.30 17:57:31.725142    RADIO_FSM     E_WAIT_DONE / S_POR_W frr:00000000
2017.04.30 17:57:31.725547   RADIO_ACTION   power up <Events=E_WAIT_DONE>
2017.04.30 17:57:31.727792    RADIO_CMD     power_up_cmd_s       020103003d0900
   Container({'xtal_options': Container({'txcO': 

In [24]:
t.display(reply)

[[3001,
  '2017.04.30 16:32:30.033303',
  ' -5080.453108',
  'RADIO_CMD',
  '  dump_radio:283 get_property:484',
  " get_property_cmd_s Container({'num_props': 7, 'cmd': 'GET_PROPERTY', 'group': 'MODEM', 'start_prop': 75})"],
 [3002,
  '2017.04.30 16:32:30.034413',
  ' -5080.451998',
  'RADIO_RSP',
  '  dump_radio:283 get_property:485',
  " get_property_rsp_s Container({'data': [12, 37, 0, 64, 3, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'cts': 255})"],
 [3003,
  '2017.04.30 16:32:30.036468',
  ' -5080.449942',
  'RADIO_CMD',
  '  dump_radio:283 get_property:484',
  " get_property_cmd_s Container({'num_props': 8, 'cmd': 'GET_PROPERTY', 'group': 'SYNTH', 'start_prop': 0})"],
 [3004,
  '2017.04.30 16:32:30.037570',
  ' -5080.448840',
  'RADIO_RSP',
  '  dump_radio:283 get_property:485',
  " get_property_rsp_s Container({'data': [44, 14, 11, 4, 12, 115, 3, 5, 0, 0, 0, 0, 0, 0, 0, 0], 'cts': 255})"],
 [3005,
  '2017.04.30 16:32:30.039280',
  ' -5080.447130',
  'RADIO_CMD',
  '  dump_radio:283 get

## code snippets

In [14]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def fetch_and_decode(group):
    gname = radio_config_group_ids.decoding[group]
    g_s = radio_config_groups[radio_config_group_ids.build(gname)]
    gid = radio_config_group_ids.build(gname)
    p = si446x_dbus_get_property(radio, gname, 0, g_s.sizeof())
    print(gname, len(p), hexlify(p))
    return g_s.parse(p)

print(interact(fetch_and_decode, group=radio_config_group_ids.encoding))

Widget Javascript not detected.  It may not be installed or enabled properly.


<function fetch_and_decode at 0x6fe662f0>


In [25]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from binascii import unhexlify
from si446x.si446xdef import *

def f(x=0x12):
    z = radio_config_groups[radio_config_group_ids.build(radio_config_group_ids.decoding[x])].parse
    print(z)

interact(f, x=radio_config_group_ids.encoding)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.f>

In [26]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display as IPdisplay
w = widgets.Dropdown(
    options = radio_config_group_ids.encoding,
    value = 2,
    description = 'Group',
)
IPdisplay(w)
print(w.value)

Widget Javascript not detected.  It may not be installed or enabled properly.


2


# Compare config files

In [27]:
from si446x.si446xdef import *
for gp_n, gp_s in radio_config_groups.iteritems():
    p = si446x_dbus_get_property(radio, radio_config_group_ids.parse(gp_n), 0, gp_s.sizeof())
    print(radio_config_group_ids.parse(gp_n), len(p), hexlify(p))
    print(gp_s.parse(p))

INT_CTL 4 033b2b00
Container:
    enable = Container:
        CHIP_INT_STATUS_EN = False
        MODEM_INT_STATUS_EN = True
        PH_INT_STATUS_EN = True
    ph_pend = Container:
        FILTER_MATCH = False
        FILTER_MISS = False
        PACKET_SENT = True
        PACKET_RX = True
        CRC_ERROR = True
        TX_FIFO_ALMOST_EMPTY = True
        RX_FIFO_ALMOST_FULL = True
    modem_pend = Container:
        POSTAMBLE_DETECT = False
        INVALID_SYNC = True
        RSSI_JUMP = False
        RSSI = True
        INVALID_PREAMBLE = False
        PREAMBLE_DETECT = True
        SYNC_DETECT = True
    chip_pend = Container:
        CAL = False
        FIFO_UNDERFLOW_OVERFLOW_ERROR = False
        STATE_CHANGE = False
        CMD_ERROR = False
        CHIP_READY = False
        LOW_BATT = False
        WUT = False
GLOBAL 10 52001860000001600000
Container:
    xo_tune = 82
    clk_cfg = Container:
        divided_clk_en = 'DISABLE'
        divided_clk_sel = 'DIV_1'
        clk_32k

RX_HOP 66 0401000102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f303132333435363738393a3b3c3d3e3f
Container:
    control = 4
    table_size = 1
    table_entries = '\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./0123456789:;<=>?'
MATCH 12 000000000000000000000000
Container:
    match_field_s = [
        Container:
            value = 0
            mask = 0
            ctrl = 0
        Container:
            value = 0
            mask = 0
            ctrl = 0
        Container:
            value = 0
            mask = 0
            ctrl = 0
        Container:
            value = 0
            mask = 0
            ctrl = 0
    ]


# Send packets as fast as possible

In [28]:
from __future__ import print_function
from time import sleep

length=10
num=2
pwr = 5
delay = .5

buf = bytearray('\x00' * length)
buf[0] = length
for i in range(1,num+1):
    buf[1] = i % 256
    rsp = radio.send(buf, pwr)
    sleep(delay)
    reply = radio.status()
#    print(reply)
    print('.', end='')

..

## Other

In [29]:
print(radio.status())

ON, <States=S_RX_ON>, unshuts 0
 RX: packets 0, len_errors 0, timeouts 0, sync_errors 0, crc_errors 0, rssi 0, 
 TX: packets 0, errors 0, timeouts 2, power 5, 
 sync frr: 08200c48
 node P211, version 0.1.53 [2017.04.30 17:57:30.366967]


In [30]:
print(radio.clear_status())

ON, <States=S_RX_ON>, unshuts 0
 RX: packets 0, len_errors 0, timeouts 0, sync_errors 0, crc_errors 0, rssi 0, 
 TX: packets 0, errors 0, timeouts 2, power 5, 
 sync frr: 08200c48
 node P211, version 0.1.53 [2017.04.30 17:57:30.420233]
 node P211, version 0.1.53 [2017.04.30 17:57:30.420510]


In [31]:
radio.control('TURNOFF')
sleep(1)
radio.control('TURNON')
sleep(1)
print(radio.status())

ON, <States=S_RX_ON>, unshuts 0
 RX: packets 0, len_errors 0, timeouts 0, sync_errors 0, crc_errors 0, rssi 0, 
 TX: packets 0, errors 0, timeouts 0, power 5, 
 sync frr: 08000052
 node P211, version 0.1.53 [2017.04.30 17:57:32.736677]
